# Readme

> **Update Jan 6, 2020**
>
> Ex 1-5 目前已经完成校对。
> Ex 6 没有校对。

- 题目按照由易到难的顺序排列，最后几题真的比较有挑战性。

- 每题我们先给出代码，再给出代码解析。

- 为了方便浏览，对于每一题的答案我们只会输出**前5行**。对应在代码中，我们会在所有代码的最后加上`[1:5]`，例如`dt[1:5]`。

- 为了充分发挥`data.table`的优点，我们争取所有题目都*只用一行代码解答*。当然这样做会一定程度上导致代码可读性变差，不利于初学者理解。我们在公众号会提供分多行解答的版本。

- 如果一题有多个答案，我们会分别列出。

- 欢迎大家关注我们的微信公众号**大猫的R语言课堂**。

- 如有任何问题请在我们的Github主页开issue。最后感谢renkun！

# Import Data

In [10]:
# We only need two packages here
library(data.table)
library(stringr)

# set `data_path` to your dir
data_path <- "C:/Users/rossz/OneDrive/App/R/stock-datatable-key/data"
setwd(data_path)

# read into data
data <- readRDS("stock-market-data.rds")
data[1:5] # show top 5 obs

symbol,date,pre_close,open,high,low,close,volume,amount,adj_factor,capt,index_w50,index_w300,index_w500,industry
<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
600000.SH,20120104,8.49,8.54,8.56,8.39,8.41,34201379,290229551,6.655275,125500555680,0.04640928,0.02125936,0,BANKS
600000.SH,20120105,8.41,8.47,8.82,8.47,8.65,132116203,1144753023,6.655275,129082022192,0.04640928,0.02125936,0,BANKS
600000.SH,20120106,8.65,8.63,8.78,8.62,8.71,61778687,537043761,6.655275,129977388820,0.04640928,0.02125936,0,BANKS
600000.SH,20120109,8.71,8.72,8.99,8.68,8.95,80136249,711429611,6.655275,133558855331,0.04640928,0.02125936,0,BANKS
600000.SH,20120110,8.95,8.95,9.10,8.88,9.07,72004632,647206633,6.655275,135349588587,0.04640928,0.02125936,0,BANKS


- 数据集为“面板数据”：包含多个股票（横截面），而每个股票则有多个按照日期排序的变量（时间序列）

- 股票代码`symbol` 和日期`date`共同组成了数据集的key，也即每个唯一的`symbol` 和`date`组合决定了一个唯一的观测。

- 整个数据集首先按照代码`symbol`排列，其次按照日期`date`排列。

- 若干主要变量说明：
    - `symbol`：股票代码。.SH 结尾的是沪股，.SZ 结尾的是深股
    - `date`：日期
    - `pre_close`： 昨收盘
    - `open`：开盘价
    - `high`：最高价（日内）
    - `low`：最低价（日内）
    - `close`：收盘价
    - `volume`：成交量
    - `amount`：成交金额
    - `industry`：行业

# Answer Keys

## 1. 哪些股票的代码中包含"8"这个数字？

首先，我们需要把股票代码`symbol`中包含8的那些观测找出来。我们可以借助`stringr`这个字符串处理包。这一步不难，稍微有些挑战的是去重。如果我们不去重，那么我们会得到非常多的重复观测。例如股票600128，如果它一共有100天的观测，那么我们会出现100个重复结果。为了去重，我们需要借助于`data.table`中的`unique`函数。代码如下。

In [12]:
data[str_detect(symbol, "8"), unique(symbol)
    ][1:5]

[1] "600008.SH" "600018.SH" "600028.SH" "600038.SH" "600048.SH"

- `str_detect`函数来自`stringr`包，它的输入是一个char vector，输出则是boolean vector，长度与原向量相同。`str_detect(symbol, "8")`含义为：对于`symbol`向量，判断其是否含有字符8，如果有，则为True，否则Faulse。

- `unique`：找出`symbol`中不重复的值。

- 在`data.table`的语法中，**先选择行，再对列进行处理**。所以上述语句会先执行`str_detect`，再执行`unique`。

## 2. 每天上涨和下跌的股票各有多少？

这一题需要引入分组的概念，并且按照“先分组，后统计”两步走。首先按照题意，我们需要为每个交易日`date`建立一个“组”。其次，对于每个组，我们需要生成两个统计数字：一个统计上涨的个数，一个统计下跌的个数。从下面的结果preview中可以看到，每个`date`都对应了两个观测，一个是“UP”，一个是“DOWN”。

In [4]:
data[, 
     .(num = uniqueN(symbol)), 
     keyby = .(date, updown = ifelse(close - pre_close > 0, "UP", "DOWN"))
    ][1:5]

date,updown,num
20120104,DOWN,2129
20120104,UP,191
20120105,DOWN,2188
20120105,UP,132
20120106,DOWN,879


- 代码第一行只有一个逗号。这是因为`data.table`的第一个语句用来对列进行选择，由于我们这里需要对所有列进行统计，所以不需要进行任何操作。

- `keyby`用来进行分组，是整个代码的核心。先来看`keyby = .(date, updown)`这个结构，他的意思是，把整个数据集按照`date`和`updown`两个变量进行分组，并依次排序。其中，`updown`是我们新建的字符变量，用来表示分组，它只取两个值：UP, DOWN。这其中的难点是建立`updown`这个变量。我们使用了`ifelse`这个函数。`ifelse(close - pre_close > 0, "UP", "DOWN")`的意思是，如果今天的收盘价高于昨天的收盘价，那么取值UP，反之取值DOWN。

- 代码第二行生成了一个新变量`num`。由于在`keyby`语句中我们已经按照日期与涨跌进行了分组，所以这一步我们只需要统计每个组有多少个股票就可以了。我们在这里使用了`uniqueN`这个函数。它是`data.table`内置函数之一，和`unique`几乎执行相同的操作，唯一不同的是，`unique`返回的是不重复的item（是一个向量），而`uniqueN`返回的是不重复的数量（是一个数字）。

- 整个代码的执行顺序是：**先选择行（逗号空白行），再分组（`keyby`语句），最后进行组间统计（`num`语句）**。

- 我们的答案中，行、列以及分组三条语句各占一行，实际上这仅仅是为了让代码更直观。如果你愿意，`data.table`允许你把所有的代码都写在同一行，就像这样：

In [ ]:
data[, .(num = uniqueN(symbol)), keyby = .(date, updown = ifelse(close - pre_close > 0, "UP", "DOWN"))]

## 3. 每天每个交易所上涨、下跌的股票各有多少？

这题和`Ex-2`非常类似，唯一的不同就是分组变量多了一个：对于每个交易日，我们不仅需要根据涨跌`updown`分组，还要根据交易所分组。也即，每个交易日都会产生四个subgroup：`SH-DOWN`, `SH-UP`, `SZ-DOWN`, `SZ-UP`.

由于股票代码`symbol`的最后两个字符表示交易所（例如，`600123.SH`表示上海交易所，股票代码600123），我们在建立分组变量时需要使用`str_sub`函数截取最后两个字符。

In [5]:
data[, .(num = uniqueN(symbol)), 
     keyby = .(date, 
               exchange = str_sub(symbol, start = -2, end = -1), 
               updown = ifelse(close - pre_close > 0, "UP", "DOWN"))
    ][1:5]

date,exchange,updown,num
20120104,SH,DOWN,836
20120104,SH,UP,85
20120104,SZ,DOWN,1293
20120104,SZ,UP,106
20120105,SH,DOWN,847


- 在`keyby`语句中，我们创建了三个分组变量，首先是日期`date`，其次是交易所`exchange`（只取SH/SZ两个值），最后是涨跌`updown`。注意这三个变量的先后顺序非常重要，不能颠倒。

- 字符串截取函数`str_sub`来自`stringr`包。`str_sub(symbol, start = -2, end = -1)`的意思是截取`symbol`最后两个字符（注意`start`/`end`取了负值）。

## 4. 沪深300成分股中，每天上涨、下跌的股票各有多少？

本题仍旧是`Ex-2`的拓展，只不过要求我们进行行选择操作。在`data.table`的`dt[i,j,by]`语法中，`i`代表行选择操作。

为了选出沪深300成分股，我们需要用到`index_w300`这个变量。`index_w300`表示一个股票在沪深300指数中的权重，如果大于零，说明它是成分股；如果为零，说明不是成分股。

In [6]:
data[index_w300 > 0, 
     .(num = uniqueN(symbol)), 
     keyby = .(date, updown = ifelse(close - pre_close > 0, "UP", "DOWN"))
    ][1:5]

date,updown,num
20120104,DOWN,280
20120104,UP,20
20120105,DOWN,250
20120105,UP,50
20120106,DOWN,98


- 为了选出沪深300成分股，我们使用了`index_w300 > 0`这个语句。`index_w300`是一个数值变量，与零进行比较运算后会生成一列与原向量等长的**布尔向量**（例如 `c(True, False False, True...)`）。`data.table`只会选择为`True`的那些元素。

- 在`data.table`的`dt[i,j,by]`语法中，先执行行选择操作`i`, 再执行分组操作`by`, 最后执行列操作`j`。因此上述代码的执行顺序为：先`index_w300 > 0`，然后`keyby`, 最后生成`num`.

## 5. 每天每个行业各有多少只股票？

仍旧是`Ex-2`的拓展，而且更为简单。我们只需要按照`date`和`industry`进行分组，然后统计每个subgroup的个数即可。

In [16]:
data[, .(stk_num = uniqueN(symbol)), 
     keyby = .(date, industry)
    ][1:5]

date,industry,stk_num
20120104,AERODEF,10
20120104,AIRLINE,12
20120104,AUTO,85
20120104,BANKS,16
20120104,BEV,30


- 我们生成一个新的变量`stk_num`用来表示每天每个行业的股票数。

## 6. 股票数最大的行业和总成交额最大的行业是否总是同一个行业？

### Key 1

In [8]:
data1 <- data[, .(trd_amount = sum(amount), stk_num = uniqueN(symbol)), 
    keyby = .(date, industry)]
data1[1:5]

date,industry,trd_amount,stk_num
20120104,AERODEF,493331236,10
20120104,AIRLINE,359576492,12
20120104,AUTO,2299263905,85
20120104,BANKS,3612012715,16
20120104,BEV,2946962800,30


In [ ]:
result <- data1[data1[, .I[trd_amount == max(trd_amount)], by = date]$V1]$industry == data1[data1[, .I[stk_num == max(stk_num)], by = date]$V1]$industry

result[1:5]

### Key 2

In [23]:
data[, .(trd_amount = sum(amount), stk_num = uniqueN(symbol)), keyby = .(date, industry)
    ][, .SD[trd_amount == max(trd_amount) & stk_num == max(stk_num), .(industry)], 
      keyby = .(date)
    ][1:5]

date,industry
20120104,HDWRSEMI
20120106,HDWRSEMI
20120213,HDWRSEMI
20120216,HDWRSEMI
20120217,HDWRSEMI


## 7. 每天涨幅超过5%、跌幅超过5%的股票各有多少？

这一题的关键思路还是`Ex-2`中的分组。首先，我们自然要对日期分组，然后按照`updown`进行分组。`updown`是用户新建的变量，只取`up5%+`和`down5%+`两个值，一个表示涨幅超过5%，一个表示跌幅超过5%。最后，我们统计每个subgroup的个数

In [7]:
data[, ':='(ret = (close - pre_close)/pre_close)
    ][ret > 0.05 | ret < -0.05, 
     .(symbol_amount = uniqueN(symbol)), 
     keyby = .(date, updown = ifelse(ret > 0.05, "up5%+", "down5%+"))
    ][1:5]

date,updown,symbol_amount
20120104,down5%+,277
20120104,up5%+,17
20120105,down5%+,885
20120105,up5%+,10
20120106,down5%+,66


- 为了方便计算，我们首先在原数据集中新增一个变量`ret`，表示股票的日收益率。`':='(ret = (close - pre_close)/pre_close)`即为新建收益率的语句，其定义为今日收盘价减去昨日收盘价，再除以昨日收盘价

- `ret > 0.05 | ret < -0.05`用来删选出收益率超过5%或小于-5%的观测。注意以上运算的结果是一个取值为`True`或`False`的向量，`data.table`最终会挑选出为`True`的那些行。

- 我们仍旧使用`ifelse`函数生成`updown`这个变量。`ifelse(ret > 0.05, "up5%+", "down5%+")`的意思是，如果条件（`ret > 0.05`）成立，那么取值`up5%+`，否则取值`down5%+`.

## 8. 每天涨幅前10的股票的总成交额和跌幅前10的股票的总成交额比例是多少？

首先，我们需要按照日期进行分组，然后对于每一个交易日，我们需要三个统计值：
- `top10`：涨幅前十的股票的总成交额
- `bottom10`：跌幅前十的股票的总成交额
- `ratio`：`top10/bottom10`

In [11]:
data[, ':='(ret = (close - pre_close)/pre_close)
    ][order(date, ret)
    ][, .(top10 = sum(amount[1:10]), bottom10 = sum(amount[(.N-10):.N])),
      keyby = date
    ][, ':='(ratio = top10/bottom10)
    ][1:5]

date,top10,bottom10,ratio
20120104,456926638,937764644,0.4872509
20120105,383022701,787215168,0.4865540
20120106,703279194,1097633048,0.6407234
20120109,558059840,1638363635,0.3406203
20120110,2948836472,1676319162,1.7591140


- 原数据集中，成交量对应变量`amount`。
- 第一步，我们为数据集新增一个变量`ret`，表示股票的日收益，对应的语句为`':='(ret = (close - pre_close)/pre_close)`
- 其次，我们先按照时间`date`，再按照收益率`ret`从低到高排序，这是为了方便后续计算，对应语句为`order(date, ret)`。其中，`order`是`data.table`中的快速排序函数。
- 接着，我们按照日期**分组**，然后计算每一天涨幅前十和跌幅前十的股票的成交额。`amount[1:10]`的意思是取`amount`的前十个观测，`amount[(.N-10):.N]`则表示`amount`的倒数10个观测。由于上一步中我们已经按照收益率`ret`进行了排序，所以`amount[1:10]`就对应了*`ret`最小的10个股票它们对应的成交额*。
- `sum`为求和函数。

## 9. 每天开盘涨停的股票与收盘涨停的股票各有多少？（涨停按照收益率超过1.5%的标准计算）

### key 1

还是老样子，首先按照`date`进行分组，然后找出每个交易日中收盘涨停与开盘涨停的股票，最后统计个数。只是题目中把涨停定义为1.5%，似乎和平日里说的10%不一致。不过不影响做题，题目怎么说我们怎么做就行。

In [13]:
data[, ':='(ret_open = open/pre_close - 1, 
            ret_close = close/pre_close - 1)
    ][, .(n_openlimit = sum(ret_open > 0.015),
          n_closelimit = sum(ret_close > 0.015)),
      keyby = date
    ][1:5]

date,n_openlimit,n_closelimit
20120104,325,70
20120105,27,60
20120106,56,743
20120109,73,2142
20120110,95,2125


- line 1-2: 首先我们在原数据集中新增两个变量`ret_open`与`ret_close`，分别表示开盘涨幅与收盘涨幅。
- line 3-5: 接着我们按照日期`date`进行分组(`keyby`语句)，然后在每组之内建立两个新的变量`n_openlimit`与`n_closelimit`用来统计开收盘涨停的股票数。这里**比较有技巧的是`sum(ret_open > 0.015)`语句**。首先我们看`ret_open > 0.015`，他的意思是把`ret_open`这个向量的**每一个分量**与1.5%进行比较，如果成立返回`True`，否则`False`。这种比较的最终的结果仍旧是一个向量。`sum(ret_open > 0.015)`则是把这个向量的所有分量相加，要知道`True`在数值计算中被转化成`1`，`False`则转化为`0`，因此`sum(ret_open > 0.015)`最终做的事情是*统计`ret_open`这个向量中有多少个分量是大于0.015的*。
- 以上技巧是非常实用的，你也可以自己验证一下，`sum(c(True, False, False))`的结果是不是等于`1`？

### Key 2 (村长的答案)

In [9]:
data[(close/pre_close - 1) > 0.015 | (open/pre_close - 1) > 0.015, .SD, 
     by = .(date, symbol, tag = ifelse((close/pre_close - 1) > 0.015, "closelimit", "openlimit"))
    ][, uniqueN(symbol), keyby = .(tag, date)
    ][1:5]

tag,date,V1
closelimit,20120104,70
closelimit,20120105,60
closelimit,20120106,743
closelimit,20120109,2142
closelimit,20120110,2125


## 10. 每天统计最近3天出现过开盘涨停的股票各有多少只？

为了避免歧义，我们首先定义最近三天如下：如果今天是`t`日，那么最近三天则为`[t-3, t-2, t-1]`这三天。当然读者可以有不同理解，这并不影响解题。

这题是`Ex-9`的衍生，且具有一定难度，难点在于我们需要进行滚动计算：当处于`t`日时，我们需要取到`n_openlimit[(t-3):(t-1)]`的值（注意`n_openlimit`在`Ex-9`中已经求得，它表示的是每日开收盘涨停的股票数）。我们使用一个非常巧妙与高效的组内循环来解决它。

In [13]:
data_ex9 <- data[, ':='(ret_open = open/pre_close - 1, 
            ret_close = close/pre_close - 1)
    ][, .(n_openlimit = sum(ret_open > 0.015),
          n_closelimit = sum(ret_close > 0.015)),
      keyby = date]

data_ex9[, 
     .(date, 
       n_openlimit_3d = {
           l = vector()
           for (t in 4:.N) {
               l[t] = sum(n_openlimit[(t-3):(t-1)])
           }
           l
       })
    ][1:5]

date,n_openlimit_3d
20120104,NA
20120105,NA
20120106,NA
20120109,408
20120110,156


- line 1-9 重复了`Ex-9`的代码，因为我们的计算需要依赖于`Ex-9`的结果
- line 8-15 是关键。先不看大括号中的部分，line 8-15的结构是这样的：
    ```
    .(date, n_openlimit={...})
    ```
    我们可以看到，这一步的目的是新建两个变量，一个是`date`，它是原数据集就有的，我们直接拿过来；另一个是`n_openlimit_3d`，它是我们在这一步新建的，表示过去三天开盘涨停的股票数。（答案只计算了开盘涨停股票数，但是读者可以自行计算收盘涨停股票数，它们的代码非常相似。）
- 我们接着来看`n_openlimit_3d`是如何建立的（大括号中的部分）。首先我们建立一个空的向量（`l = vector()`， line 10），它的每个元素都是`NA`。我们将会**填充**这个空向量`l`，最终让它变成我们的目标向量`n_openlimit_3d`。
- line 11-13 是具体的填充过程。因为我们需要向前追溯三天，也就是说，最开头的三天是不会产生有效值的，只有第四天开始才会有值。因此，我们只需要从第四个元素起填充，一直填到最后一个元素，这就是`for (t in 4:.N)`所做的事。其中`.N`是`data.table`自带的特殊变量，表示这个数据集的长度。
- line 12 是最关键的一行。首先通过 line 11 的循环，我们得到了一个变量`t`，它表示*当前我们所处的天数*。其次，我们使用`n_openlimit[(t-3):(t-1)]`来获得`[(t-3):(t-1)]`区间的开盘涨停股票数。我们可以看到，`n_openlimit[(t-3):(t-1)]`的结果将是**一个长度为三向量**，每个分量表示某一天的开盘涨停股票数。我们最后使用`sum`对其求和，就得到了过去三天涨停股票的总数。
- line 14是为了语法要求而存在的。读者可以把它理解为一个`return`语句：它告诉R，在一系列的操作之后，大括号（line 11-13）最终的输出将会是`l`这个向量。

### Key 2 （村长的答案）

In [27]:
data[(close/pre_close - 1) > 0.015 | (open/pre_close - 1) > 0.015, .SD, by = .(date, symbol, tag = ifelse((close/pre_close - 1) > 0.015, "closelimit", "openlimit"))
    ][, uniqueN(symbol), keyby = .(tag, date)
    ][, .(symbol_amount = {
        a <- vector()
        for (i in 4:.N) {
            a[i] <- sum(V1[(i-3):(i-1)])
        }
        a
    }, date), keyby = .(tag)
    ][1:5]

tag,symbol_amount,date
closelimit,NA,20120104
closelimit,NA,20120105
closelimit,NA,20120106
closelimit,873,20120109
closelimit,2945,20120110


## 11. 股票每天的成交额变化率和收益率的相关性如何？

In [35]:
data[, .(amount_change = {
    a <- vector()
    for (i in 2:.N) {
        a[i] <- amount[i]/amount[i-1] - 1
    }
    a
    }, ret = close/pre_close - 1, symbol = symbol)
    ][is.finite(amount_change), na.omit(.SD)
    ][, cor(amount_change, ret)]

[1] 0.163331

## 12. 每天每个行业的总成交额变化率和行业收益率的相关性如何？

In [34]:
data[, .(ind_amount = sum(amount), weight = capt/sum(capt), ret = close/pre_close - 1), keyby = .(date, industry)
    ][, .(ind_ret = sum(weight * ret), ind_amount = ind_amount), keyby = .(industry, date)
    ][, unique(.SD)
    ][, .(ind_amount_change = {
        a <- vector()
        for (i in 2:.N) {
            a[i] <- ind_amount[i]/ind_amount[i-1] - 1
        }
        a
    }, ind_ret = ind_ret), keyby = .(industry)
    ][!is.na(ind_amount_change), cor(ind_amount_change, ind_ret)]

[1] 0.3942165

## 13. 每天市场的总成交额变化率和市场收益率相关性如何？

In [33]:
data[, .(mkt_amount = sum(amount), weight = capt/sum(capt), ret = close/pre_close - 1), keyby = date
    ][, .(mkt_ret = sum(weight * ret), mkt_amount = mkt_amount), keyby = date
    ][, unique(.SD)
    ][, .(mkt_amount_change = {
        a <- vector()
        for (i in 2:.N) {
            a[i] <- mkt_amount[i]/mkt_amount[i-1]-1
        }
        a
    }, mkt_ret = mkt_ret)
    ][!is.na(mkt_amount_change), cor(mkt_amount_change, mkt_ret)]

[1] 0.4424592

## 14. 每天市场的总成交额的变化率和所有股票收益率的标准差相关性如何？

In [32]:
data[, .(mkt_amount = sum(amount), ret = close/pre_close - 1, symbol = symbol), keyby = date
    ][, .(ret_sd = unique(sd(ret)), mkt_amount = unique(mkt_amount)), keyby = date
    ][, .(mkt_amount_change = {
        a <- vector()
        for (i in 2:.N) {
            a[i] <- mkt_amount[i]/mkt_amount[i-1] - 1
        }
        a
    }, ret_sd = ret_sd)
    ][!is.na(mkt_amount_change), cor(mkt_amount_change, ret_sd)]

[1] -0.1913893

## 15. 每天每个行业的总成交额变化率和行业内股票收益率的标准差相关性如何？

In [36]:
data[, .(ind_amount = sum(amount), ret = close/pre_close - 1, symbol = symbol), keyby = .(industry, date)
    ][, .(ind_ret_sd = unique(sd(ret)), ind_amount = unique(ind_amount)), keyby = .(industry, date)
    ][, .(ind_amount_change = {
        a <- vector()
        for (i in 2:.N) {
            a[i] <- ind_amount[i]/ind_amount[i-1] - 1
        }
        a
    }, ind_ret_sd = ind_ret_sd), keyby = industry
    ][!is.na(ind_amount_change), cor(ind_ret_sd, ind_amount_change)]

[1] 0.1043456

## 16. 上证50、沪深300、中证500指数成分股中，沪股和深股各有多少？

In [7]:
data[, melt(.SD, measure.vars = patterns("index"), variable.name = "index_name")
    ][value > 0, .(stkcd_amount = uniqueN(symbol)), by = .(index_name, type = ifelse(str_detect(symbol, "SH"), "SH", "SZ"))]

index_name,type,stkcd_amount
<fct>,<chr>,<int>
index_w50,SH,54
index_w300,SH,205
index_w300,SZ,95
index_w500,SH,297
index_w500,SZ,248


## 17. 上证50、沪深300、中证500指数成分股中，行业分布如何？

In [11]:
data[, melt(.SD, measure.vars = patterns("index"), variable.name = "index_name")
    ][value > 0, .(stkcd_amount = uniqueN(symbol)), by = .(index_name, industry)
    ][1:5]

index_name,industry,stkcd_amount
<fct>,<chr>,<int>
index_w50,BANKS,11
index_w50,MTLMIN,10
index_w50,ENERGY,7
index_w50,DVFININS,6
index_w50,MACH,4


## 18. 每天上证50、沪深300、中证500指数成分股的总成交额各是多少？

In [14]:
data[, melt(.SD, measure.vars = patterns("index"), variable.name = "index_name")
    ][value > 0, .(amount = sum(amount)), by = .(index_name, date)
    ][1:5]

index_name,date,amount
<fct>,<int>,<dbl>
index_w50,20120104,11372037211
index_w50,20120105,15174601949
index_w50,20120106,11457969686
index_w50,20120109,19783087939
index_w50,20120110,25840613156


## 19. 上证50、沪深300、中证500指数日收益率的历史波动率是多少？

In [18]:
data[, melt(.SD, measure.vars = patterns("index"), variable.name = "index_name")
    ][, .(index_ret = sum(value * (close/pre_close - 1))), by = .(date, index_name)
    ][, .(vol = sd(index_ret)), by = .(index_name)]

index_name,vol
<fct>,<dbl>
index_w50,0.01234205
index_w300,0.01336797
index_w500,0.01607531


## 20. 上证50、沪深300、中证500指数日收益率的相关系数矩阵？

In [42]:
data[, .(index_w50_ret = sum(index_w50 * (close/pre_close - 1)), index_w300_ret = sum(index_w300 * (close/pre_close - 1)), index_w500_ret = sum(index_w500 * (close/pre_close - 1))), keyby = date
    ][, cor(.SD[, -1])]

,index_w50_ret,index_w300_ret,index_w500_ret
index_w50_ret,1.0000000,0.9760849,0.8476975
index_w300_ret,0.9760849,1.0000000,0.9333580
index_w500_ret,0.8476975,0.9333580,1.0000000


## 21. 上证50、沪深300、去除上证50的沪深300指数日收益率的相关系数矩阵？

In [32]:
data[, .(index_w50_ret = sum(index_w50 * (close/pre_close - 1)), index_w300_ret = sum(index_w300 * (close/pre_close - 1)), index_w300_50 = ifelse(index_w50 == 0 & index_w300 > 0, index_w300, 0), close = close, pre_close = pre_close), keyby = date       ][, .(index_w50_ret = unique(index_w50_ret), index_w300_ret = unique(index_w300_ret), index_w300_50_ret = sum(index_w300_50 * (close/pre_close - 1)) %>% unique()), keyby = date
    ][, cor(.SD[, -1])]

,index_w50_ret,index_w300_ret,index_w300_50_ret
index_w50_ret,1.0000000,0.9760849,0.9294381
index_w300_ret,0.9760849,1.0000000,0.9874054
index_w300_50_ret,0.9294381,0.9874054,1.0000000


## 22. 每天沪深300指数成分占比最大的10只股票是哪些？

In [43]:
data[order(date, index_w300), .(symbol = symbol[1:10]), by = .(date)
    ][1:5]

date,symbol
20120104,600004.SH
20120104,600006.SH
20120104,600007.SH
20120104,600011.SH
20120104,600012.SH


## 23. 各个行业的平均每日股票数量从大到小排序是什么？

In [44]:
data[, .(stkcd_amount = uniqueN(symbol)), keyby = .(date, industry)
    ][order(date, -stkcd_amount)
    ][1:5]

date,industry,stkcd_amount
20120104,HDWRSEMI,224
20120104,CHEM,219
20120104,MACH,201
20120104,HEALTH,172
20120104,ELECEQP,131


## 24. 每个行业每天成交额最大的一只股票代码是什么？

In [45]:
data[order(-amount), .(symbol = symbol[1]), keyby = .(date, industry)
    ][1:5]

date,industry,symbol
20120104,AERODEF,000768.SZ
20120104,AIRLINE,600029.SH
20120104,AUTO,600104.SH
20120104,BANKS,600036.SH
20120104,BEV,600519.SH


## 25. 每个行业每天最大成交额是最小成交额的几倍？

In [46]:
data[order(-amount) & amount > 0, .(times = amount[1]/amount[.N]), 
     keyby = .(date, industry)
    ][1:5]

date,industry,times
20120104,AERODEF,1.6822610
20120104,AIRLINE,17.0340980
20120104,AUTO,0.3134478
20120104,BANKS,4.5180305
20120104,BEV,0.2922401


## 26. 每个行业每天成交额最大的5只股票和成交额总和是多少？

In [47]:
data[order(-amount), .(symbol = symbol[1:5], amount = sum(amount[1:5])),
     keyby = .(date, industry)
    ][1:5]

date,industry,symbol,amount
20120104,AERODEF,000768.SZ,384758246
20120104,AERODEF,600316.SH,384758246
20120104,AERODEF,002025.SZ,384758246
20120104,AERODEF,600893.SH,384758246
20120104,AERODEF,600118.SH,384758246


## 27. 每个行业每天成交额超过该行业中股票成交额80%分位数的股票的平均收益率是多少？

In [48]:
data[, .(symbol = symbol, ret = close/pre_close - 1, amount = amount, industry, date)
    ][, .SD[amount > quantile(amount, 0.8)], keyby = .(date, industry)
    ][, .(aver_ret = mean(ret)), keyby = .(date, industry)
    ][1:5]

date,industry,aver_ret
20120104,AERODEF,0.005714392
20120104,AIRLINE,-0.024726379
20120104,AUTO,-0.017813417
20120104,BANKS,-0.007846674
20120104,BEV,-0.056254460


## 28. 每天成交额最大的10%的股票的平均收益率和成交额最小的10%的股票的平均收益率的相关系数是多少？

In [49]:
data[, .(symbol = symbol, ret = close/pre_close - 1, amount = amount, date)
    ][, .(ret_aver = mean(ret)), keyby = .(date, tag = ifelse(amount > quantile(amount, 0.9), "max10%"，ifelse(amount < quantile(amount, 0.1), "min10%", "others")))
    ][, .SD[-3], keyby = .(date)
    ][1:5]

date,tag,ret_aver
20120104,max10%,-0.018105729
20120104,min10%,-0.017907674
20120105,max10%,-0.021758776
20120105,min10%,-0.030488091
20120106,max10%,0.005994575


## 29. 每天哪些行业的平均成交额高于全市场平均成交额？

In [50]:
data[, .(amount_aver = mean(amount), industry = industry, amount = amount), keyby = .(date)
    ][, .(amount_ind_aver = mean(amount), amount_aver), keyby = .(date, industry)
    ][amount_ind_aver > amount_aver, unique(.SD[, 1:2])
    ][1:5]

date,industry
20120104,AERODEF
20120104,BANKS
20120104,BEV
20120104,CONMAT
20120104,DVFININS


## 30. 每天每个股票对市场的超额收益率是多少？

In [51]:
data[, .(stkcd_ret = close/pre_close - 1, weight = capt/sum(capt), symbol), keyby = date
    ][, .(mkt_ret = sum(weight*stkcd_ret), stkcd_ret, symbol), keyby = date
    ][, .(alpha = coef(lm(stkcd_ret ~ mkt_ret))[1], beta = coef(lm(stkcd_ret ~ mkt_ret))[2], mkt_ret, stkcd_ret, symbol, date)
    ][, .(abnr_ret = stkcd_ret - alpha - beta * mkt_ret), keyby = .(date, symbol)
    ][1:5]

date,symbol,abnr_ret
20120104,000001.SZ,-0.007575707
20120104,000002.SZ,0.001264443
20120104,000004.SZ,-0.002243615
20120104,000005.SZ,0.020006076
20120104,000006.SZ,0.010249978


## 31. 每天每个股票对市场去除自身的超额收益率是多少？

In [52]:
data[, .(stkcd_ret = close/pre_close - 1, symbol, weight =  capt/(sum(capt) - capt)),keyby = date
    ][, .(mkt_ret = sum(weight*stkcd_ret), stkcd_ret, symbol), keyby = date
    ][, .(alpha = coef(lm(stkcd_ret ~ mkt_ret))[1], beta = coef(lm(stkcd_ret ~ mkt_ret))[2], mkt_ret, stkcd_ret, symbol, date)
    ][, .(abnr_ret = stkcd_ret - alpha - beta * mkt_ret), keyby = .(date, symbol)
    ][1:5]

date,symbol,abnr_ret
20120104,000001.SZ,-0.007762946
20120104,000002.SZ,0.001077204
20120104,000004.SZ,-0.002430854
20120104,000005.SZ,0.019818837
20120104,000006.SZ,0.010062739


## 32. 每天每个股票对行业的超额收益率是多少？

In [53]:
data[, .(stkcd_ret = close/pre_close - 1, symbol, weight = capt/sum(capt)), keyby = .(industry,date)
    ][, .(ind_ret = sum(weight*stkcd_ret), stkcd_ret, symbol), keyby = .(industry, date)
    ][, .(alpha = coef(lm(stkcd_ret ~ ind_ret))[1], beta = coef(lm(stkcd_ret ~ ind_ret))[2], ind_ret, stkcd_ret, symbol, date), keyby = .(industry)
    ][, .(abnr_ret = stkcd_ret - alpha - beta * ind_ret), keyby = .(date, symbol)
    ][1:5]

date,symbol,abnr_ret
20120104,000001.SZ,-0.0193325437
20120104,000002.SZ,0.0009002525
20120104,000004.SZ,0.0070727953
20120104,000005.SZ,0.0196418857
20120104,000006.SZ,0.0098857881


## 33. 每天每个股票对行业去除自身的超额收益率是多少？

In [54]:
data[, .(stkcd_ret = close/pre_close - 1, symbol, weight = capt/(sum(capt) - capt)), keyby = .(industry,date)
    ][, .(ind_ret = sum(weight*stkcd_ret), stkcd_ret, symbol), keyby = .(industry, date)
    ][, .(alpha = coef(lm(stkcd_ret ~ ind_ret))[1], beta = coef(lm(stkcd_ret ~ ind_ret))[2], ind_ret, stkcd_ret, symbol, date), keyby = .(industry)
    ][, .(abnr_ret = stkcd_ret - alpha - beta * ind_ret), keyby = .(date, symbol)
    ][1:5]

date,symbol,abnr_ret
20120104,000001.SZ,-0.0196847604
20120104,000002.SZ,0.0006789611
20120104,000004.SZ,0.0071187283
20120104,000005.SZ,0.0194205943
20120104,000006.SZ,0.0096644967


## 34. 每个股票每天对市场的超额收益率与对行业的超额收益率的相关系数如何？

In [55]:
mkt <- data[, .(stkcd_ret = close/pre_close - 1, weight = capt/sum(capt), symbol), keyby = date
    ][, .(mkt_ret = sum(weight/100*stkcd_ret), stkcd_ret, symbol), keyby = date
    ][, .(alpha = coef(lm(stkcd_ret ~ mkt_ret))[1], beta = coef(lm(stkcd_ret ~ mkt_ret))[2], mkt_ret, stkcd_ret, symbol, date)
    ][, .(abnr_mkt_ret = stkcd_ret - alpha - beta * mkt_ret), keyby = .(date, symbol)]
ind <- data[, .(stkcd_ret = close/pre_close - 1, symbol, weight = capt/sum(capt)), keyby = .(industry,date)
    ][, .(ind_ret = sum(weight/100*stkcd_ret), stkcd_ret, symbol), keyby = .(industry, date)
    ][, .(alpha = coef(lm(stkcd_ret ~ ind_ret))[1], beta = coef(lm(stkcd_ret ~ ind_ret))[2], ind_ret, stkcd_ret, symbol, date), keyby = .(industry)
    ][, .(abnr_ind_ret = stkcd_ret - alpha - beta * ind_ret), 
      keyby = .(date, symbol)]

cor(mkt[ind, on = .(date, symbol)][, 3:4])
rm(mkt, ind)

,abnr_mkt_ret,abnr_ind_ret
abnr_mkt_ret,1.000000,0.946769
abnr_ind_ret,0.946769,1.000000


## 35. 每天有哪些行业的平均收益率超过市场平均收益率？

In [56]:
data[, .(stkcd_ret = close/pre_close - 1, symbol, ind_weight = capt/sum(capt), capt), keyby = .(industry,date)
    ][, .(ind_ret = sum(ind_weight*stkcd_ret), stkcd_ret, symbol, capt), keyby = .(industry, date)
    ][, .(ind_ret, mkt_weight = capt/sum(capt), stkcd_ret, industry), keyby = date
    ][, .(ind_ret, mkt_ret = sum(mkt_weight*stkcd_ret), industry), keyby = date
    ][ind_ret > mkt_ret, unique(.SD)
    ][1:5]

date,ind_ret,mkt_ret,industry
20120104,-0.008403388,-0.01581648,AERODEF
20120104,-0.013057659,-0.01581648,AUTO
20120104,-0.006737250,-0.01581648,BANKS
20120104,-0.003713831,-0.01581648,ENERGY
20120104,-0.011340144,-0.01581648,HOUSEDUR


## 36. 每天每个行业对市场的超额收益率是多少？

In [57]:
data[, .(stkcd_ret = close/pre_close - 1, symbol, ind_weight = capt/sum(capt), capt), keyby = .(industry,date)
    ][, .(ind_ret = sum(ind_weight*stkcd_ret), stkcd_ret, symbol, capt), keyby = .(industry, date)
    ][, .(ind_ret, mkt_weight = capt/sum(capt), stkcd_ret, industry), keyby = date
    ][, .(ind_ret, mkt_ret = sum(mkt_weight*stkcd_ret), industry), keyby = date
    ][, unique(.SD)
    ][, .(alpha = coef(lm(ind_ret ~ mkt_ret))[1], beta = coef(lm(ind_ret ~ mkt_ret))[2], ind_ret, mkt_ret, industry, date)
    ][, .(abnr_ret = ind_ret - alpha - beta*mkt_ret, date), by = industry
    ][1:5]

industry,abnr_ret,date
AERODEF,0.009899362,20120104
AERODEF,-0.008739172,20120105
AERODEF,-0.028247448,20120106
AERODEF,-0.006283080,20120109
AERODEF,0.005342716,20120110


## 37. 每天每个行业对去除本行业后的市场超额收益是多少？

In [58]:
data[, .(stkcd_ret = close/pre_close - 1, symbol, ind_weight = capt/sum(capt), capt), keyby = .(industry,date)
    ][, .(ind_ret = sum(ind_weight*stkcd_ret), stkcd_ret, symbol, capt), keyby = .(industry, date)
    ][, .(ind_capt = sum(capt), ind_ret), keyby = .(industry, date)
    ][, unique(.SD)
    ][, .(ind_mkt_weight = ind_capt/(sum(ind_capt)-ind_capt), ind_ret, industry), keyby = date
    ][, .(mkt_ret = sum(ind_mkt_weight * ind_ret), ind_ret, industry), by = date
    ][, .(alpha = coef(lm(ind_ret ~ mkt_ret))[1], beta = coef(lm(ind_ret ~ mkt_ret))[2], ind_ret, mkt_ret, industry, date)
    ][, .(abnr_ret = ind_ret - alpha - beta*mkt_ret, date), by = industry
    ][1:5]

industry,abnr_ret,date
AERODEF,0.009421348,20120104
AERODEF,-0.010103267,20120105
AERODEF,-0.028697841,20120106
AERODEF,-0.006553974,20120109
AERODEF,0.005790573,20120110


## 38. 每天分别有多少股票是最近连续3个交易日上涨、下跌的？

In [59]:
data[, .(stkcd_ret = close/pre_close - 1), keyby = .(symbol, date)
    ][, {
        l <- list()
        b1 <- stkcd_ret > 0
        b2 <- stkcd_ret < 0
        for (t in 4:.N) {
            l[[t]] <- as.list(c(r3day_up = mean(c(b1[t-1], b1[t-2], b1[t-3])), r3day_dn = mean(c(b2[t-1], b1[t-2], b1[t-3])), date = date[t]))
        }
        rbindlist(l)
    }    
    , keyby = .(symbol)
    ][!is.na(date), .(stkcd_amount = uniqueN(symbol)), keyby = .(date, tag = ifelse(r3day_up == 1, "r3day_up", ifelse(r3day_dn == 1, "r3day_dn", "others")))
    ][tag == "r3day_dn"|tag == "r3day_up"
    ][1:5]

date,tag,stkcd_amount
20120109,r3day_dn,17
20120109,r3day_up,16
20120110,r3day_dn,1
20120110,r3day_up,71
20120111,r3day_dn,6


## 39. 每天分别有多少股票是最近连续3个交易日收益率超过当天市场平均收益率？

In [60]:
data[, .(stkcd_ret = close/pre_close - 1, mkt_weight = capt/sum(capt), symbol), keyby = .(date)
    ][, .(mkt_ret = sum(stkcd_ret * mkt_weight), symbol, stkcd_ret), keyby = .(date)
    ][, .(r3day_ret = ifelse(stkcd_ret > mkt_ret, 1, 0)), keyby = .(symbol, date)
    ][, {
        l <- list()
        for (t in 4:.N) {
            l[[t]] <- as.list(c(r3day_ret = mean(c(r3day_ret[t-1], r3day_ret[t-2], r3day_ret[t-3])), date = date[t]))
        }
        rbindlist(l)
    }, keyby = symbol
    ][r3day_ret == 1 & !is.na(date), .(stkcd_amount = uniqueN(symbol)),
      keyby = date
    ][1:5]

date,stkcd_amount
20120109,81
20120110,95
20120111,785
20120112,905
20120113,392


## 40. 每天分别有多少股票是最新5个交易日中至少有4个交易日的收益率超过当天市场平均收益率？

In [61]:
data[, .(stkcd_ret = close/pre_close - 1, mkt_weight = capt/sum(capt), symbol), keyby = .(date)
    ][, .(mkt_ret = sum(stkcd_ret * mkt_weight), symbol, stkcd_ret), keyby = .(date)
    ][, .(r3day_ret = ifelse(stkcd_ret > mkt_ret, 1, 0)), keyby = .(symbol, date)
    ][, {
        l <- list()
        for (t in 6:.N) {
            l[[t]] <- as.list(c(r3day_ret = mean(c(r3day_ret[t-1], r3day_ret[t-2], r3day_ret[t-3], r3day_ret[t-4], r3day_ret[t-5])), date = date[t]))
        }
        rbindlist(l)
    }, keyby = symbol
    ][r3day_ret > 0.8 & !is.na(date), .(stkcd_amount = uniqueN(symbol)), keyby = date
    ][1:5]

date,stkcd_amount
20120111,14
20120112,22
20120113,184
20120116,48
20120117,18


## 41. 每个月中，个股月收益超过市场月收益1倍以上的股票有哪些？

In [62]:
data[, .(stkcd_ret = close/pre_close - 1, mkt_weight = capt/sum(capt), symbol), keyby = .(date)
    ][, .(mkt_ret = sum(mkt_weight * stkcd_ret), stkcd_ret, symbol, date_ym = str_sub(date, start = 1, end = 6)), keyby = .(date)
    ][, .(stkcd_m_ret = mean(stkcd_ret), date, mkt_ret), keyby = .(symbol, date_ym)
    ][, .(stkcd_m_ret, symbol, date, mkt_m_ret = mean(mkt_ret)), keyby = .(date_ym)
    ][stkcd_m_ret > 2*mkt_m_ret, .(symbol = unique(symbol)), keyby = .(date_ym)
    ][1:5]

date_ym,symbol
201201,000009.SZ
201201,000012.SZ
201201,000017.SZ
201201,000018.SZ
201201,000030.SZ


## 42. 每个月中，个股月收益超过行业月收益1倍以上的股票有哪些？

In [63]:
data[, .(stkcd_ret = close/pre_close - 1, ind_weight = capt/sum(capt), symbol), keyby = .(industry, date)
    ][, .(ind_ret = sum(ind_weight * stkcd_ret), stkcd_ret, symbol, date_ym = str_sub(date, start = 1, end = 6)), keyby = .(industry, date)
    ][, .(stkcd_m_ret = mean(stkcd_ret), date, ind_ret), keyby = .(symbol, date_ym)
    ][, .(stkcd_m_ret, symbol, date, ind_m_ret = mean(ind_ret)), keyby = .(date_ym)
    ][stkcd_m_ret > 2*ind_m_ret, .(symbol = unique(symbol)), keyby = .(date_ym)
    ][1:5]

date_ym,symbol
201201,000001.SZ
201201,000006.SZ
201201,000007.SZ
201201,000009.SZ
201201,000012.SZ


## 43. 每个股票的收益率对市场收益率的相关系数最高的10个股票是哪些？

In [64]:
data[, .(stkcd_ret = close/pre_close - 1, mkt_weight = capt/sum(capt), symbol), keyby = .(date)
    ][, .(stkcd_ret, mkt_ret = sum(mkt_weight * stkcd_ret), symbol), keyby = .(date)
    ][, .(cor_coef = cor(stkcd_ret, mkt_ret)), keyby = symbol
    ][order(-cor_coef), .SD[1:10]
    ][1:5]

Warning message in cor(stkcd_ret, mkt_ret):
"the standard deviation is zero"Warning message in cor(stkcd_ret, mkt_ret):
"the standard deviation is zero"Warning message in cor(stkcd_ret, mkt_ret):
"the standard deviation is zero"Warning message in cor(stkcd_ret, mkt_ret):
"the standard deviation is zero"Warning message in cor(stkcd_ret, mkt_ret):
"the standard deviation is zero"Warning message in cor(stkcd_ret, mkt_ret):
"the standard deviation is zero"Warning message in cor(stkcd_ret, mkt_ret):
"the standard deviation is zero"Warning message in cor(stkcd_ret, mkt_ret):
"the standard deviation is zero"Warning message in cor(stkcd_ret, mkt_ret):
"the standard deviation is zero"Warning message in cor(stkcd_ret, mkt_ret):
"the standard deviation is zero"Warning message in cor(stkcd_ret, mkt_ret):
"the standard deviation is zero"Warning message in cor(stkcd_ret, mkt_ret):
"the standard deviation is zero"Warning message in cor(stkcd_ret, mkt_ret):
"the standard deviation is zero"Warning mess

symbol,cor_coef
300331.SZ,1.0000000
600508.SH,0.9279554
601101.SH,0.9097758
000685.SZ,0.8979148
601666.SH,0.8939529


## 44. 每个行业日收益率的历史波动率是多少？（用日收益率计算标准差）

In [65]:
data[, .(stkcd_ret = close/pre_close - 1, ind_weight = capt/sum(capt), symbol), keyby = .(industry, date)
    ][, .(ind_ret = sum(stkcd_ret * ind_weight)), keyby = .(industry, date)
    ][, .(ind_vol = sd(ind_ret)), keyby = .(industry)
    ][1:5]

industry,ind_vol
AERODEF,0.017499519
AIRLINE,0.017127041
AUTO,0.015096513
BANKS,0.008105859
BEV,0.016407569


## 45. 各个行业的日收益率的相关系数矩阵如何？哪两个行业相关性最高、最低？

In [66]:
cor.coef <- data[, .(stkcd_ret = close/pre_close - 1, ind_weight = capt/sum(capt), symbol, capt), keyby = .(industry, date)
    ][, .(ind_ret = sum(stkcd_ret * ind_weight)), keyby = .(industry, date)
    ][,  dcast(.SD, date ~ industry, value.var = "ind_ret")
    ][, cor(.SD[, -1])]

which(max(cor.coef[cor.coef != 1]) == cor.coef, arr.ind = TRUE)

which(min(cor.coef) == cor.coef, arr.ind = TRUE)

rm(cor.coef)

,row,col
HDWRSEMI,16,13
ELECEQP,13,16


,row,col
BEV,5,4
BANKS,4,5


## 46. 各个行业的收益率对市场收益率的相关系数由高到低排列如何？

In [67]:
data[, .(stkcd_ret = close/pre_close - 1, symbol, ind_weight = capt/sum(capt), capt), keyby = .(industry,date)
    ][, .(ind_ret = sum(ind_weight*stkcd_ret), stkcd_ret, symbol, capt), keyby = .(industry, date)
    ][, .(ind_ret, mkt_weight = capt/sum(capt), stkcd_ret, industry), keyby = date
    ][, .(ind_ret, mkt_ret = sum(mkt_weight*stkcd_ret), industry), keyby = date
    ][, unique(.SD)
    ][, .(cor_coef = cor(ind_ret, mkt_ret)), keyby = .(industry)
    ][order(-cor_coef)
    ][1:5]

industry,cor_coef
TRDDIST,0.9519013
MACH,0.9511527
CHEM,0.9455305
ELECEQP,0.9451663
CNSTENG,0.9446049


## 47. 每个月总成交额比上个月下降幅度最大的行业是哪个？

In [70]:
data[, .(date_ym = str_sub(date, start = 1, end = 6), amount), keyby = .(industry, date)
    ][, .(ind_m_amount = sum(amount)), keyby = .(industry, date_ym)
    ][, .(dn_m_range = {
        a <- vector()
        for (t in 2:.N) {
            a[t] <- ind_m_amount[t] - ind_m_amount[t-1]
        }
        a 
    }, date_ym = date_ym), keyby = industry
    ][!is.na(dn_m_range), .SD[min(dn_m_range) == dn_m_range & dn_m_range < 0], keyby = date_ym]

date_ym,industry,dn_m_range
201203,CONMAT,-21752549725
201204,MTLMIN,-165775634730
201205,REALEST,-4528375663
201206,MTLMIN,-123271994315


## 48. 数据当中各个股票的最大回撤幅度是多少？（最大回撤是从一个高点到低点的降幅的最大值）

In [71]:
data[, .(max_rtrt = max(high) - min(low)), keyby = symbol
    ][1:5]

symbol,max_rtrt
000001.SZ,3.33
000002.SZ,2.43
000004.SZ,2.93
000005.SZ,1.74
000006.SZ,3.00


## 49. 每只股票的胜率是多少？（胜率是每天收益率为正数的概率）

In [72]:
data[, .(stkcd_ret = close/pre_close - 1), keyby = .(symbol, date)
    ][, .(gain_ratio = sum(stkcd_ret > 0)/.N), keyby = symbol
    ][1:5]

symbol,gain_ratio
000001.SZ,0.4188034
000002.SZ,0.4700855
000004.SZ,0.5128205
000005.SZ,0.1538462
000006.SZ,0.5128205


## 50. 每只股票的盈亏比是多少？（盈亏比是正收益之和与负收益之和的比值的绝对值）

In [73]:
data[, .(stkcd_ret = close/pre_close - 1), keyby = .(symbol, date)
    ][, .(stkcd_ret_pn = sum(stkcd_ret)), keyby = .(symbol, tag = ifelse(stkcd_ret > 0, "gain", "loss"))
    ][, .(gl_ratio = abs(stkcd_ret_pn[1]/stkcd_ret_pn[.N])), keyby = symbol
    ][1:5]

symbol,gl_ratio
000001.SZ,0.9718614
000002.SZ,1.2796856
000004.SZ,1.0828690
000005.SZ,0.5532427
000006.SZ,1.3719934


## 51. 市场的胜率是多少？（市场收益率为正的概率）

In [75]:
data[, .(stkcd_ret = close/pre_close - 1, mkt_weight = capt/sum(capt)), keyby = date
    ][, .(mkt_ret = sum(stkcd_ret * mkt_weight)), keyby = date
    ][, .(gain_ratio = sum(mkt_ret > 0)/.N)]

gain_ratio
0.4871795


## 52. 市场的盈亏比是多少？（市场中每个股票的市值加权正收益和市值加权负收益之比）

In [76]:
data[, .(stkcd_ret = close/pre_close - 1, mkt_weight = capt/sum(capt)), keyby = date
    ][, .(mkt_ret = sum(stkcd_ret * mkt_weight)), keyby = date
    ][, .(mkt_ret_pn = sum(mkt_ret)), keyby = .(tag = ifelse(mkt_ret > 0, "gain", "loss"))
    ][, .(gl_ratio = abs(mkt_ret_pn[1]/mkt_ret_pn[.N]))]

gl_ratio
1.147373


## 53. 每个行业的胜率是多少？

In [77]:
data[, .(stkcd_ret = close/pre_close - 1, ind_weight = capt/sum(capt)), keyby = .(industry, date)
    ][, .(ind_ret = sum(stkcd_ret * ind_weight)), keyby = .(industry, date)
    ][, .(gain_ratio = sum(ind_ret > 0)/.N), keyby = industry
    ][1:5]

industry,gain_ratio
AERODEF,0.5213675
AIRLINE,0.4786325
AUTO,0.4700855
BANKS,0.4615385
BEV,0.5470085


## 54. 每个行业的盈亏比是多少？（行业盈亏比是行业内每个股票的市值加权的正收益率和市值加权的负收益率之比）

In [78]:
data[, .(stkcd_ret = close/pre_close - 1, ind_weight = capt/sum(capt)), keyby = .(industry, date)
    ][, .(ind_ret = sum(stkcd_ret * ind_weight)), keyby = .(industry, date)
    ][, .(ind_ret_pn = sum(ind_ret)), keyby = .(industry, tag = ifelse(ind_ret > 0, "gain", "loss"))
    ][, .(gl_ratio = abs(ind_ret_pn[1]/ind_ret_pn[.N])), keyby = industry
    ][1:5]

industry,gl_ratio
AERODEF,0.9973617
AIRLINE,1.0715381
AUTO,1.1206090
BANKS,1.0158729
BEV,1.2812672


## 55. 是否存在股票的月成交额超过所在行业当月中某天一天总成交额的情况？

In [79]:
data[, .(date_ym = str_sub(date, start = 1, end = 6), amount, industry), keyby = .(symbol, date)
    ][, .(stkcd_m_amount = sum(amount), amount, industry, date), keyby = .(date_ym, symbol)
    ][, .(stkcd_m_amount, ind_d_amount = sum(amount), date_ym, symbol), keyby = .(industry, date)
    ][stkcd_m_amount > ind_d_amount
    ][1:5]

industry,date,stkcd_m_amount,ind_d_amount,date_ym,symbol
AERODEF,20120104,1460111191,493331236,201201,000768.SZ
AERODEF,20120104,976641005,493331236,201201,002025.SZ
AERODEF,20120104,1125117546,493331236,201201,600118.SH
AERODEF,20120104,1522650678,493331236,201201,600316.SH
AERODEF,20120104,675067956,493331236,201201,600879.SH


## 56. 每天每个行业编入、编出的股票各有多少？

In [80]:
data[, .(symbol_list = list(symbol)), keyby = .(industry, date)
    ][, {
        l <- list()
        for (i in 2:.N) {
            intec <- intersect(symbol_list[[i]], symbol_list[[i-1]]) %>% length()
            len1 <- length(symbol_list[[i]])
            len2 <- length(symbol_list[[i-1]])
            l[[i]] <- list(export = len2-intec, import = len1-intec)
        }
        rbindlist(l)
    }, keyby = industry
    ][1:5]

industry,export,import
AERODEF,0,0
AERODEF,0,0
AERODEF,0,0
AERODEF,0,0
AERODEF,0,0


## 57. 每天每个行业内股票收益率的标准差是多少？

In [81]:
data[, .(stkcd_ret = close/pre_close - 1), keyby = .(symbol, industry, date)
    ][, .(ret_sd = sd(stkcd_ret)), keyby = .(industry, date)
    ][1:5]

industry,date,ret_sd
AERODEF,20120104,0.02836880
AERODEF,20120105,0.02902021
AERODEF,20120106,0.03193141
AERODEF,20120109,0.01377493
AERODEF,20120110,0.01417229


## 58. 每天每个行业内股票收益率的标准差的相关性如何？

In [85]:
data[, .(stkcd_ret = close/pre_close - 1), keyby = .(symbol, industry, date)
    ][, .(ret_sd = sd(stkcd_ret)), keyby = .(industry, date)
    ][, dcast(.SD, date ~ industry, value.var = "ret_sd")
    ][, cor(.SD[, -1])
    ][1:5, 1:5]

,AERODEF,AIRLINE,AUTO,BANKS,BEV
AERODEF,1.00000000,0.11523031,0.33195634,0.08021524,0.02497865
AIRLINE,0.11523031,1.00000000,0.22235548,0.02446198,0.02250268
AUTO,0.33195634,0.22235548,1.00000000,0.07283962,0.26374233
BANKS,0.08021524,0.02446198,0.07283962,1.00000000,0.06073737
BEV,0.02497865,0.02250268,0.26374233,0.06073737,1.00000000


## 59. 每天计算出成交额的 z-score （减去均值除以标准差）, 该指标能解释下一天个股超额收益率的多少比例？

In [87]:
data[, .(stkcd_ret = close/pre_close - 1, weight = capt/sum(capt), symbol, amount), keyby = date
    ][, .(mkt_ret = sum(weight*stkcd_ret), stkcd_ret, symbol, amount), keyby = date
    ][, .(alpha = coef(lm(stkcd_ret ~ mkt_ret))[1], beta = coef(lm(stkcd_ret ~ mkt_ret))[2], mkt_ret, stkcd_ret, symbol, date, amount)
    ][, .(abnr_ret = stkcd_ret - alpha - beta * mkt_ret, amount), keyby = .(date, symbol)
    ][, .(abnr_ret, zscore = (amount - mean(amount))/ sd(amount), symbol), keyby = date
    ][, .(abnr_lead_ret = shift(abnr_ret, n = 1L, type = "lead"), zscore, symbol), keyby = symbol
    ][, summary(lm(abnr_lead_ret ~ zscore))$r.squared]

[1] 0.0003313487

## 60. 每个股票的收益率和300、500指数收益率可以回归出一个截距项和2个beta，这两个beta的分布如何？

In [88]:
data[, .(stkcd_ret = close/pre_close - 1, ind_w300_ret = sum((close/pre_close - 1) * index_w300), ind_w500_ret = sum((close/pre_close - 1) * index_w500), symbol), keyby = date
    ][, .(coef_beta = coef(lm(stkcd_ret ~ ind_w300_ret + ind_w500_ret))[-1], tag = c("ind_w300_ret", "ind_w500_ret")), keyby = symbol
    ][!is.na(coef_beta), .(sample_mean = mean(coef_beta), sample_sd = sd(coef_beta)), keyby = tag]

tag,sample_mean,sample_sd
ind_w300_ret,-0.1583057,0.9317869
ind_w500_ret,1.1264276,0.8311509


## 61. 每天开盘后到最高价涨幅最大的100只股票同样也是全天(昨收到今收)涨幅最大的100只股票的比例是多少?

In [89]:
data[, .(high_open = high/open - 1, clo_pre = close/pre_close - 1), by = .(symbol, date)
    ][order(date, -high_open), .(ho_symbol = list(symbol[1:100])), 
      keyby = date
    ][data[, .(high_open = high/open - 1, clo_pre = close/pre_close - 1), by = .(symbol, date)    
        ][order(date, -clo_pre), .(cp_symbol = list(symbol[1:100])), 
          keyby = date], on = .(date)
    ][, {
        l <- list()
        for (i in 1:.N) {
            l[[i]] <- list(intec = intersect(cp_symbol[[i]], ho_symbol[[i]]))
        }
        rbindlist(l)
    }, keyby = date
    ][, .(ratio = .N/100), keyby = date
    ][1:5]

date,ratio
20120104,0.52
20120105,0.52
20120106,0.61
20120109,0.70
20120110,0.59


## 62. 每天计算最近三天每天对市场的超额收益率都排进当天前100的股票有哪些?

In [90]:
data[, .(stkcd_ret = close/pre_close - 1, weight = capt/sum(capt), symbol), keyby = date
    ][, .(mkt_ret = sum(weight*stkcd_ret), stkcd_ret, symbol), keyby = date
    ][, .(alpha = coef(lm(stkcd_ret ~ mkt_ret))[1], beta = coef(lm(stkcd_ret ~ mkt_ret))[2], mkt_ret, stkcd_ret, symbol, date)
    ][, .(abnr_ret = stkcd_ret - alpha - beta * mkt_ret), keyby = .(date, symbol)
    ][order(date, -abnr_ret), .(symbol = list(symbol[1:100])), keyby = date
    ][, {
        l <- list()
        for (t in 4:.N) {
            l[[t]] <- list(symbol = Reduce(intersect, list(symbol[[t]], symbol[[t-1]], symbol[[t-2]])), date = date[[t]])
        }
        rbindlist(l)
    }][1:5]

Warning message in rbindlist(l):
"Column 1 ['symbol'] of item 4 is length 0. This (and 10 others like it) has been filled with NA (NULL for list columns) to make each item uniform."

symbol,date
NA,20120109
000791.SZ,20120110
002118.SZ,20120111
600971.SH,20120111
000791.SZ,20120111


## 63. 每天计算最近三天每天对行业的超额收益率都排进当天行业前30%的股票有哪些?

In [91]:
data[, .(stkcd_ret = close/pre_close - 1, symbol, weight = capt/sum(capt)), keyby = .(industry,date)
    ][, .(ind_ret = sum(weight*stkcd_ret), stkcd_ret, symbol), keyby = .(industry, date)
    ][, .(alpha = coef(lm(stkcd_ret ~ ind_ret))[1], beta = coef(lm(stkcd_ret ~ ind_ret))[2], ind_ret, stkcd_ret, symbol, date), keyby = .(industry)
    ][, .(abnr_ret = stkcd_ret - alpha - beta * ind_ret), keyby = .(date, symbol)
    ][order(date, -abnr_ret) & abnr_ret > quantile(abnr_ret, 0.7), .(symbol = list(symbol)), keyby = date
    ][, {
        l <- list()
        for (t in 4:.N) {
            l[[t]] <- list(symbol = Reduce(intersect, list(symbol[[t]], symbol[[t-1]], symbol[[t-2]])), date = date[[t]])
        }
        rbindlist(l)
    }][1:5]

symbol,date
000039.SZ,20120109
000045.SZ,20120109
000411.SZ,20120109
000422.SZ,20120109
000518.SZ,20120109
